# 必須課題(1) 動作の確認
> このページで用いた検索結果に対するpyNTCIREVALの出力のうち, MSnDCG@0003とnERR@0003が
> 講義資料の定義に従った計算と一致していることを確かめよ.
> つまり, nDCG@3とnERR@3を計算するプログラムを書き,
> その結果がpyNTCIREVALの結果と一致していることを確認せよ.

In [1]:
import numpy as np
import pandas as pd
from functools import reduce
from operator import add, mul

In [2]:
# [START 講義で扱ったnDCGとnERRの実装]
def DCG(rels):
    """DCGの計算
    
    params
    ---------
    rels : list, i番目の要素はi番目の文書の適合度を表す.
    """
    elems = [(2 ** rel - 1) / np.log2(1 + (idx + 1)) for idx, rel in enumerate(rels)]
    return reduce(add, elems)

def ERR(rels):
    MAX_REL = 2.0
    stop_proba = lambda rel: (2 ** rel - 1) / (2 ** MAX_REL)
    ans = 0.0
    continue_probas = [1.0 - stop_proba(rel) for rel in rels]
    for idx, rel in enumerate(rels):
        p_stop = stop_proba(rel)
        rec = 1.0 if idx == 0 else reduce(mul, continue_probas[:idx])
        ans += (p_stop * rec) / (idx + 1)
    return ans

In [3]:
# [START 文書と関連度の読み込み] pandas が扱いやすい
df = pd.read_csv("../data/eval/q1.rel", header=None, sep=" ")
df.columns = ["doc", "rel"]
res = ["d1", "d2", "d3"]
gain_map = { "L0": 0, "L1": 1, "L2": 2 } # 授業ではL2: 3だが、計算が合わない...
df["score"] = df["rel"].map(lambda rel: gain_map[rel])

In [4]:
# [START nDCG@3の計算]
rels = df[df["doc"].isin(["d1", "d2", "d3"])]["score"].tolist()
opt_rels = df["score"].sort_values(ascending=False).tolist()[0:3]
dcg_obs = DCG(rels)
dcg_opt = DCG(opt_rels)

print("DCG_obs: {}".format(dcg_obs))
print("DCG_opt: {}".format(dcg_opt))

ndcg = dcg_obs / dcg_opt

print("nDCG@3: {}".format(round(ndcg, 4))) # 丸め込むことで一致する

DCG_obs: 2.5
DCG_opt: 4.130929753571458
nDCG@3: 0.6052


In [5]:
# [START ERR@3の計算]
err_obs = ERR(rels)
err_opt = ERR(opt_rels)
nerr = err_obs / err_opt

print("ERR_obs: {}".format(err_obs))
print("ERR_opt: {}".format(err_opt))
print("nERR@3: {}".format(round(nerr, 4))) # 丸め込むことで一致する

ERR_obs: 0.4375
ERR_opt: 0.796875
nERR@3: 0.549


確かに一致していることがわかる.

# 必須課題(2) 独自データに対する評価指標の計算
> 演習課題1で扱った検索課題集合と検索結果に対して各自で評価用データを作成し
> pyNTCIREVALを用いて評価指標を計算せよ.
> そして, MRR, nDCG@3及びnERR@3の平均を報告し,
> それらの値の違いが各指標のどういった要因によるものかを考察せよ.
> なお, 演習課題1で扱ったコーパス以外で評価データを作成しても良い.
> ただし, 評価データはダミーデータではなく, 実際の何らかのランキングを評価したものとし,
> 検索課題(クエリ)は3つ以上とする.

+ 文書の適合度は上位k件について自分で決めれば良い
    + k = 10くらいが好ましい?
+ MRRはRRの平均値

In [6]:
# [START 評価用データの作成]
# q1 = {"京都", "紅葉"}
!pyNTCIREVAL label -r ../data/hw/q1.rel < ../data/hw/q1.res > ../data/hw/label.q1.rel
print("q1の上位3件と関連度")
!cat ../data/hw/label.q1.rel
# q2 = {"銀閣寺", "場所"}
!pyNTCIREVAL label -r ../data/hw/q2.rel < ../data/hw/q2.res > ../data/hw/label.q2.rel
print("q2の上位3件と関連度")
!cat ../data/hw/label.q2.rel
# q3 = {"鴨川", "料理"}
!pyNTCIREVAL label -r ../data/hw/q3.rel < ../data/hw/q3.res > ../data/hw/label.q3.rel
print("q3の上位3件と関連度")
!cat ../data/hw/label.q3.rel

q1の上位3件と関連度
d33 L2
d14 L1
d25 L1
q2の上位3件と関連度
d6 L0
d17 L1
d71 L1
q3の上位3件と関連度
d62 L0
d15 L0
d32 L1


In [7]:
# [START 評価指標の計算結果の書き出し]
!pyNTCIREVAL compute -r ../data/hw/q1.rel -g 1:3 --cutoffs=1,3 < ../data/hw/label.q1.rel > ../data/hw/q1.eval
!pyNTCIREVAL compute -r ../data/hw/q2.rel -g 1:3 --cutoffs=1,3 < ../data/hw/label.q2.rel > ../data/hw/q2.eval
!pyNTCIREVAL compute -r ../data/hw/q3.rel -g 1:3 --cutoffs=1,3 < ../data/hw/label.q3.rel > ../data/hw/q3.eval

In [8]:
!cat ../data/hw/q1.eval | grep RR
print("---"*30)
!cat ../data/hw/q2.eval | grep RR
print("---"*30)
!cat ../data/hw/q3.eval | grep RR

 RR=                 1.0000
 ERR=                0.7969
 nERR@0001=          1.0000
 nERR@0003=          0.9273
------------------------------------------------------------------------------------------
 RR=                 0.5000
 ERR=                0.1875
 nERR@0001=          0.0000
 nERR@0003=          0.2353
------------------------------------------------------------------------------------------
 RR=                 0.3333
 ERR=                0.0833
 nERR@0001=          0.0000
 nERR@0003=          0.2133


In [9]:
# [START MRRの計算]
print("MRR: {}".format(np.mean([1.0, 0.5, 0.3333])))

MRR: 0.6111


In [10]:
# [START ERRの平均値の計算]
print("nERR@3: {}".format(np.mean([0.9273, 0.2325, 0.2133])))

nERR@3: 0.4577


In [11]:
!cat ../data/hw/q1.eval | grep MSnDCG@0003
print("---" * 30)
!cat ../data/hw/q2.eval | grep MSnDCG@0003
print("---" * 30)
!cat ../data/hw/q3.eval | grep MSnDCG@0003

 MSnDCG@0003=        0.6462
------------------------------------------------------------------------------------------
 MSnDCG@0003=        0.2738
------------------------------------------------------------------------------------------
 MSnDCG@0003=        0.2346


In [12]:
# [START nDCGの平均値の計算]
print("nDCG@3: {}".format(np.mean([0.6462, 0.2738, 0.2346])))

nDCG@3: 0.38486666666666663


## 各指標の値のばらつきに対する考察
計算された結果を次のようにまとめる.

| 指標 | 平均 | q1 | q2 | q3 |
|:------|:------|:----|:----|:----|
| MRR | 0.611 | 1.000 | 0.500 | 0.333 |
| nDCG@3 | 0.384 | 0.646 | 0.273 | 0.234 |
| nERR@3 | 0.457 | 0.927 | 0.235 | 0.213 |

### MRR
`MRR`は次のような式で表現される評価指標である.
$$MRR = \frac{1}{\|Q|\|}\sum_{i=1}^{\|Q\|}\frac{1}{rank_{i}}$$
$rank_{i}$は, ランキングの結果上位から最初に関連度のある文書の順位を意味する.

クエリq1, q2, q3についてのランクづけ結果において, それぞれ1, 2, 3番目に初めて関連のある文書が現れている. 
したがって, 計算されたMRRの値から, 大抵1番目か2番目までに少なくとも関連のある文書が現れるような検索を実現できていると言える.

また, MRRの導出において考慮される関連のある文書の数はたかだか1つであり, それぞれの検索結果について3番目までには関連のある文書が現れているので, その他の指標の値に比べて高い値となった.

この性質から, MRRはその他の指標と比較して, 計算するためのコストが小さい評価指標であると言える.
なぜならば, 何らかの検索結果について, 上位から「関連のある」文書を一つ見つけるだけでよく, 判断回数が少なく, かつ関連度合まで考慮する必要がないからである.

### nDCG@3 & nERR@3
nDCG, nERRは, MRRと比較して文書の適合「度」を考慮する評価指標であり, それぞれの式は次のように表現される.
$$
nDCG@k = \frac{DCG@k_{obs}}{DCG@k_{opt}}
$$
$$
nERR@k = \frac{ERR@k_{obs}}{ERR@k_{opt}}
$$
ただし
$$
DCG@k = \sum_{i=1}^{k}\frac{2^{rel_{i}} - 1}{log_{2}(i+1)}
$$
$$
ERR@k = \sum_{i=1}^{k}\frac{1}{i}\prod_{j}^{i - 1}(1 - R_{j})R_{i}
$$

これら2つの指標を比較して, nERRは上位に適合度が大きい文書があるほどそれよりも下位の文書が指標に与える影響が小さいと言える.

q1は関連度の高い文書が上位に現れているため, nERR@3の値は0.93程度と非常に高いが, nDCGについては, 10番目までに関連度が`L2`であるような文書が他に存在するため0.64程度と小さい値になっている.
このことから, nERRは検索結果の上位に存在する関連度の大きな文書に影響を受けやすい一方で, nDCGは検索結果に含まれる文書全体の関連度の高さに影響を受けやすい指標であることがわかる.

#### 注意
nDCGとnERRについての考察は, q2, q3については検索結果がさほど好ましくなく, それぞれの指標の特徴があまり現れていないためq1に注目して考えた.